# Define Size

In [1]:
import random
sf= 100
#sf = random.randrange(1, 10)*10

StatementMeta(, 56c14dbb-fbb8-45a9-8ec4-be1701222dc4, 3, Finished, Available)

In [2]:
from psutil import *
core= cpu_count()
vCPU = str(core) + " vCPU"
mem=round(virtual_memory().total/(1024 * 1024 * 1024),0)
runtime = 'Sorting Lineitem-SF'+str(sf)+' ,Fabric Notebook, '+vCPU+' '+str(mem)+'GB'
runtime

StatementMeta(, 56c14dbb-fbb8-45a9-8ec4-be1701222dc4, 4, Finished, Available)

'Sorting Lineitem-SF100 ,Fabric Notebook, 64 vCPU 504.0GB'

# Install Packages

In [3]:
!pip install duckdb --pre --upgrade            

StatementMeta(, 56c14dbb-fbb8-45a9-8ec4-be1701222dc4, 5, Finished, Available)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 131.5 MB/s eta 0:00:0000:0100:01


In [4]:
%%time
import duckdb
import pathlib
import os
import pandas as pd
import time
from datetime import datetime
if not os.path.isdir(f'/lakehouse/default/Files/{sf}'):
    for x in range(0, sf) :
        con=duckdb.connect()
        con.sql('PRAGMA disable_progress_bar;SET preserve_insertion_order=false')
        con.sql(f"CALL dbgen(sf={sf} , children ={sf}, step = {x})") 
        for tbl in ['lineitem'] :
            pathlib.Path(f'/lakehouse/default/Files/{sf}/{tbl}').mkdir(parents=True, exist_ok=True) 
            con.sql(f"COPY (SELECT * FROM {tbl}) TO '/lakehouse/default/Files/{sf}/{tbl}/{x:03d}.parquet' ")
        con.close()

StatementMeta(, 56c14dbb-fbb8-45a9-8ec4-be1701222dc4, 6, Finished, Available)

CPU times: user 633 ms, sys: 96.3 ms, total: 730 ms
Wall time: 12 s


In [5]:
import os
mypath = f"/lakehouse/default/Files/{sf}/Sort/"
os.makedirs(mypath, exist_ok=True)
for root, dirs, files in os.walk(mypath, topdown=False):
    for file in files:
        os.remove(os.path.join(root, file))

    # Add this block to remove folders
    for dir in dirs:
        os.rmdir(os.path.join(root, dir))

StatementMeta(, 56c14dbb-fbb8-45a9-8ec4-be1701222dc4, 7, Finished, Available)

# DuckDB

In [6]:
%%time
start = time.time()
import duckdb
con=duckdb.connect()
con.sql(
f'''
PRAGMA disable_progress_bar;
COPY  (select * from '/lakehouse/default/Files/{sf}/lineitem/*.parquet' order by l_shipdate )
to '/lakehouse/default/Files/{sf}/Sort/lineitem_DuckDB_Sort' (FORMAT 'PARQUET', PER_THREAD_OUTPUT TRUE)
''')
con.close()
duckdb_duration = time.time()-start
df = pd.DataFrame([{'engine':'DuckDB', 'value':duckdb_duration , 'time':datetime.now().strftime('%Y-%m-%d %H:%M:%S') , 'sf':sf ,'cpu':core }])
sparkDf = spark.createDataFrame(df);
sparkDf.write.mode("append").format("delta").save("Tables/sort_stats")

StatementMeta(, 56c14dbb-fbb8-45a9-8ec4-be1701222dc4, 8, Finished, Available)

/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:604: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.


CPU times: user 40min 4s, sys: 1h 45min, total: 2h 25min 4s
Wall time: 5min 8s


# Spark

In [7]:
%%time
import time
start = time.time()
df = spark.read.parquet(f"Files/{sf}/lineitem/*.parquet")
df.orderBy("l_shipdate")
df.write.mode("overwrite").format("parquet").save(f"Files/{sf}/Sort/Lineitem_Spark_Sort")
Spark_duration = time.time()-start
df = pd.DataFrame([{'engine':'Spark', 'value':Spark_duration , 'time':datetime.now().strftime('%Y-%m-%d %H:%M:%S') , 'sf':sf ,'cpu':core }])
sparkDf = spark.createDataFrame(df);
sparkDf.write.mode("append").format("delta").save("Tables/sort_stats")

StatementMeta(, 56c14dbb-fbb8-45a9-8ec4-be1701222dc4, 9, Finished, Available)

/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:604: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.


CPU times: user 41 ms, sys: 8.67 ms, total: 49.6 ms
Wall time: 2min 5s
